In [1]:
import os
import json
import random
import subprocess
import pandas as pd

In [3]:
projects = ["JPetStore"]#["JPetStore", "DayTrader", "AcmeAir", "Plants"]
for proj in projects:
	with open(f"results/Mo2oM_HardClustering/Mo2oM_HardClustering_{proj}.json") as f:
		data = json.load(f)
	df = pd.DataFrame(data)
	row = df[(df["n_clusters"] == 9) & (df["alpha"] == 0.4)]
	classes_microservices = row.iloc[0]["microservices"]
	libs = os.path.join("..", "Mo2oM", "JavaParser", "lib", "javaparser-core-3.25.5-SNAPSHOT.jar") + os.pathsep + \
		   os.path.join("..", "Mo2oM", "JavaParser", "lib", "json-20230618.jar")
	json_path = os.path.join("..", "Mo2oM", "JavaParser", "classes.json")
	source_path = os.path.join("..", "test_projects", proj, "OneFileSource.java")
	parser_path = os.path.join("..", "Mo2oM", "JavaParser", "Parser.java")
	subprocess.run(['java', '-cp', libs, parser_path, source_path, json_path])
	with open(json_path, "rt") as classes_file:
		classes_info = json.load(classes_file)
	for clss in classes_info:
		for call in classes_info[clss]["method_calls"]:
			for other_clss in classes_info:
				if call["method_name"] in classes_info[other_clss]["methods"]:
					call["class_name"] = other_clss
					if clss == other_clss:
						break
	class_names = list(classes_info.keys())
	calls = []
	for caller_class_i, caller_class_n in enumerate(classes_info):
		caller_microservices = classes_microservices[caller_class_i]
		if caller_microservices != [-1]:
			for call in classes_info[caller_class_n]["method_calls"]:
				if call["class_name"] in class_names:
					callee_microservices = classes_microservices[class_names.index(call["class_name"])]
					if callee_microservices != [-1]:
						for caller_microservice in caller_microservices:
							if caller_microservice in callee_microservices:
								calls.append({
									"caller_microservice": f"microservice_{caller_microservice}",
									"caller_class": caller_class_n,
									"callee_microservice": f"microservice_{caller_microservice}",
									"callee_class": call["class_name"],
								})
							else:
								calls.append({
									"caller_microservice": f"microservice_{caller_microservice}",
									"caller_class": caller_class_n,
									"callee_microservice": f"microservice_{random.choice(callee_microservices)}",
									"callee_class": call["class_name"],
								})
df = pd.DataFrame(calls)
df

,caller_microservice,caller_class,callee_microservice,callee_class
0,microservice_6,OracleSequenceDao,microservice_6,Category
1,microservice_6,OracleSequenceDao,microservice_6,OracleSequenceDao
2,microservice_0,ViewProductController,microservice_0,PetStoreFacade
3,microservice_0,ViewProductController,microservice_0,PetStoreFacade
4,microservice_5,AddItemToCartController,microservice_7,Cart
...,...,...,...,...
400,microservice_1,OrderServiceClient,microservice_1,OrderServiceClient
401,microservice_4,SqlMapAccountDao,microservice_6,Order
402,microservice_4,SqlMapAccountDao,microservice_3,AccountActionForm
403,microservice_4,SqlMapAccountDao,microservice_3,AccountActionForm


In [4]:
df = df[(df["caller_class"] != df["callee_class"]) | (df["caller_microservice"] != df["callee_microservice"])]
df

,caller_microservice,caller_class,callee_microservice,callee_class
0,microservice_6,OracleSequenceDao,microservice_6,Category
2,microservice_0,ViewProductController,microservice_0,PetStoreFacade
3,microservice_0,ViewProductController,microservice_0,PetStoreFacade
4,microservice_5,AddItemToCartController,microservice_7,Cart
5,microservice_5,AddItemToCartController,microservice_0,PetStoreFacade
...,...,...,...,...
399,microservice_1,OrderServiceClient,microservice_6,Order
401,microservice_4,SqlMapAccountDao,microservice_6,Order
402,microservice_4,SqlMapAccountDao,microservice_3,AccountActionForm
403,microservice_4,SqlMapAccountDao,microservice_3,AccountActionForm


In [5]:
class_list = {'Account',
			  'AddItemToCartAction',
			  'BaseAction',
			  'Category',
			  'EditAccountAction',
			  'EditAccountFormAction',
			  'ListOrdersAction',
			  'ListOrdersController',
			  'NewAccountFormAction',
			  'NewOrderFormAction',
			  'OracleSequenceDao',
			  'SearchProductsAction',
			  'SendOrderConfirmationEmailAdvice',
			  'SignonAction',
			  'SignonController',
			  'SqlMapAccountDao',
			  'ViewOrderAction'}

In [6]:
new_df = df[df["caller_class"].isin(class_list) & df["callee_class"].isin(class_list)]

In [7]:
jdata = []
microservices = new_df["caller_microservice"].unique()
for ms in microservices:
	classes = new_df[new_df["caller_microservice"] == ms]["caller_class"].unique()
	for clss in classes:
		calls = new_df[(new_df["caller_microservice"] == ms) & (new_df["caller_class"] == clss)]
		imports = []
		for v in calls.values:
			if v[2] != ms or v[3] != clss:
				imports.append(f"{projects[0]}."+v[2]+"."+v[3])
		jdata.append({"name": f"{projects[0]}."+ms+"."+clss, "imports": imports})
no_imports = set()
for jd in jdata:
	for imp in jd["imports"]:
		for jd in jdata:
			if jd["name"] == imp:
				break
		else:
			no_imports.add(imp)
for ni in no_imports:
	jdata.append({"name": ni, "imports": []})
with open(f"Mo2oM_HardClustering_{projects[0]}_calls.json", "w") as f:
	json.dump(jdata, f)